In [7]:
import pandas as pd
from pathlib import Path

In [8]:
data_path = Path("../data")

In [9]:
df_withdrawals = pd.read_parquet(data_path / "zrive_advertiser_withdrawals.parquet")
df_advertiser = pd.read_parquet(data_path / "zrive_dim_advertiser.parquet")
df_monthly = pd.read_parquet(data_path / "zrive_fct_monthly_snapshot_advertiser.parquet")

In [10]:
# Withdrawals 
def add_churn(df: pd.DataFrame):
    CHURN_REASONS_EXCLUDED = [
        'Upselling-cambio de contrato',
        'Cambio a Bundle Online',
        'Cambio de Contrato/propuesta/producto'
    ]
    df["churn"] = (
        (df["withdrawal_type"] == "TOTAL") &
        (df["withdrawal_status"] != "Denegada") &
        (~df["withdrawal_reason"].isin(CHURN_REASONS_EXCLUDED))
    ).astype(int)
    return df

def convert_datetime_to_month_period(df, datetime_col, new_col, drop_original=True):
    df[new_col] = pd.to_datetime(df[datetime_col]).dt.to_period('M')
    if drop_original:
        df = df.drop(columns=[datetime_col])
    return df

def add_predict_month(df: pd.DataFrame, predict_col = "predict_month", withdrawal_col="withdrawal_month", n: int = 1):
    df[predict_col] = df[withdrawal_col] - n
    return df


In [11]:
WITHDRAWAL_COLS_TO_DROP = ["withdrawal_id", "withdrawal_status", "withdrawal_type", "withdrawal_reason", "withdrawal_month"]

df_withdrawals = convert_datetime_to_month_period(
    df_withdrawals, 
    datetime_col='withdrawal_creation_date',
    new_col='withdrawal_month',
    drop_original=True
)
df_withdrawals = add_predict_month(df_withdrawals, n=1)
df_withdrawals = add_churn(df_withdrawals)
df_withdrawals.drop(columns=WITHDRAWAL_COLS_TO_DROP, inplace=True)

In [12]:
df_withdrawals

,advertiser_zrive_id,predict_month,churn
0,257,2012-05,1
1,219,2012-05,1
2,487,2012-05,1
3,476,2012-05,1
4,452,2012-05,1
...,...,...,...
20674,5441,2024-11,1
20675,5439,2024-11,1
20676,154,2024-11,1
20677,1352,2024-11,1


### df_monthly --> como definir churn

In [13]:
df_monthly[df_monthly['advertiser_zrive_id'] == 1]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,...,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract
0,1,202301,47,47,75,18,40890.0,6,6,6,...,14,15,0,440.8,12,3,15,NaN,NaN,True
3542,1,202302,44,44,50,10,34740.0,3,3,1,...,18,6,3,464.8,5,4,9,NaN,NaN,True
6905,1,202303,39,39,50,7,11850.0,3,3,1,...,5,4,2,464.8,3,3,6,NaN,NaN,True
10328,1,202304,39,39,50,6,15930.0,3,3,1,...,7,3,2,464.8,3,3,6,NaN,NaN,True
13747,1,202305,0,0,150,8,19140.0,0,3,1,...,13,4,2,464.8,4,4,8,NaN,NaN,True


In [14]:
def convert_period_int_to_month_period(df, period_col='period_int', new_col='month_period'):
    """Convierte el formato YYYYMM a un período mensual de pandas"""
    df[new_col] = pd.to_datetime(df[period_col].astype(str) + '01', format='%Y%m%d').dt.to_period('M')
    return df

In [15]:
df_monthly = convert_period_int_to_month_period(df_monthly)

In [16]:
df_monthly.head()

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,...,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period
0,1,202301,47,47,75,18,40890.0,6,6,6,...,15,0,440.8,12,3,15,NaN,NaN,True,2023-01
1,2,202301,31,31,150,4,17970.0,10,10,4,...,2,2,75.4,2,2,4,NaN,NaN,True,2023-01
2,3,202301,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,NaN,NaN,False,2023-01
3,4,202301,79,79,85,14,27157.5,3,3,1,...,8,2,299.6,6,6,12,NaN,NaN,True,2023-01
4,6,202301,20,20,20,16,79492.5,0,0,1,...,4,3,86.5,4,11,15,NaN,NaN,True,2023-01


In [17]:
def add_churn_target(df_monthly, df_withdrawals):
    """
    Añade el target de churn a df_monthly, usando lo calculado en df_withdrawals

    Args:
        df_monthly: DataFrame con datos mensuales de anunciantes
        df_withdrawals: DataFrame procesado con información de churn
        
    Returns:
        DataFrame con columna 'churn' añadida
    """
    # Realizar la unión basada en advertiser_id y el período mensual
    df_target = df_monthly.merge(
        df_withdrawals[['advertiser_zrive_id', 'predict_month', 'churn']],
        left_on=['advertiser_zrive_id', 'month_period'],
        right_on=['advertiser_zrive_id', 'predict_month'],
        how='left'
    )

    # Rellenar los valores NaN en churn con 0 (no hubo churn)
    df_target['churn'] = df_target['churn'].fillna(0)

    # Eliminar la columna redundante predict_month
    if 'predict_month' in df_target.columns:
        df_target = df_target.drop(columns=['predict_month'])

    return df_target

In [18]:
df_target = add_churn_target(df_monthly, df_withdrawals)

In [19]:
df_target[df_target['advertiser_zrive_id'] == 1]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,...,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
0,1,202301,47,47,75,18,40890.0,6,6,6,...,0,440.8,12,3,15,NaN,NaN,True,2023-01,0.0
3553,1,202302,44,44,50,10,34740.0,3,3,1,...,3,464.8,5,4,9,NaN,NaN,True,2023-02,1.0
6948,1,202303,39,39,50,7,11850.0,3,3,1,...,2,464.8,3,3,6,NaN,NaN,True,2023-03,0.0
10383,1,202304,39,39,50,6,15930.0,3,3,1,...,2,464.8,3,3,6,NaN,NaN,True,2023-04,0.0
13816,1,202305,0,0,150,8,19140.0,0,3,1,...,2,464.8,4,4,8,NaN,NaN,True,2023-05,0.0


In [20]:
def reduce_columns(df):
    """
    Selecciona solo ciertas columnas para el análisis inicial
    """
    essential_columns = [
        'advertiser_zrive_id',
        'period_int',
        'month_period',
        'monthly_published_ads',
        'monthly_contracted_ads',
        'has_active_contract',
        'churn'
    ]

    # Asegurarse de que todas las columnas existen
    existing_columns = [col for col in essential_columns if col in df.columns]

    return df[existing_columns]

In [21]:
df_target = reduce_columns(df_target)

In [22]:
df_target[df_target['advertiser_zrive_id'] == 1]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
0,1,202301,2023-01,47,75,True,0.0
3553,1,202302,2023-02,44,50,True,1.0
6948,1,202303,2023-03,39,50,True,0.0
10383,1,202304,2023-04,39,50,True,0.0
13816,1,202305,2023-05,0,150,True,0.0


Creo que es un buen punto de partida. 

- Creo que deberiamos eliminar las filas que hay despues de la fila con churn=1

- Hay varios casos que veo que pueden darnos problemas

    - Caso 1: Las filas que tienen has_active_contract en False que significan? Yo entiendo que no tienen contrato y que por tanto no pueden tener interacciones ese mes pero si que tienen, que hacemos con estas filas las dejamos o las quitamos?


    - Caso 2: Usuarios que dejan de tener actividad pero no tienen churn confirmada en df_withdrawals entre 2023 y 2024

    - Caso 3: Usuarios con multiples churn

### Caso 1: has_active_contract en False 

In [29]:
df_target[df_target['has_active_contract'] == False].head(15)

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
2,3,202301,2023-01,0,0,False,0.0
29,41,202301,2023-01,52,100,False,0.0
55,76,202301,2023-01,20,75,False,0.0
503,693,202301,2023-01,22,35,False,0.0
574,787,202301,2023-01,16,200,False,0.0
585,803,202301,2023-01,13,20,False,0.0
661,917,202301,2023-01,27,35,False,0.0
775,1078,202301,2023-01,75,75,False,0.0
897,1246,202301,2023-01,15,20,False,0.0
952,1333,202301,2023-01,34,35,False,0.0


In [30]:
#Caso 1: Ejemplo

df_target[df_target['advertiser_zrive_id'] == 2353]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
1685,2353,202301,2023-01,45,400,False,0.0
5137,2353,202302,2023-02,400,400,False,0.0
8527,2353,202303,2023-03,400,400,True,0.0
11932,2353,202304,2023-04,400,400,True,1.0
15360,2353,202305,2023-05,426,400,True,0.0
18742,2353,202306,2023-06,290,400,True,0.0
22129,2353,202307,2023-07,300,400,True,0.0
25501,2353,202308,2023-08,300,400,False,0.0


In [ ]:
#Caso 1: Ejemplo

df_target[df_target['advertiser_zrive_id'] == 76]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
55,76,202301,2023-01,20,75,False,0.0
3605,76,202302,2023-02,29,75,False,0.0
7002,76,202303,2023-03,25,75,False,0.0
10433,76,202304,2023-04,14,75,True,1.0
13866,76,202305,2023-05,29,150,True,0.0


In [28]:
df_target[df_target['advertiser_zrive_id'] == 787]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
574,787,202301,2023-01,16,200,False,0.0
30992,787,202310,2023-10,20,20,True,0.0
34162,787,202311,2023-11,20,20,True,0.0
37392,787,202312,2023-12,20,20,True,0.0
40618,787,202401,2024-01,35,35,True,0.0
43836,787,202402,2024-02,35,35,True,0.0
47092,787,202403,2024-03,35,35,True,0.0
50422,787,202404,2024-04,35,35,True,1.0
53802,787,202405,2024-05,20,20,True,0.0
57264,787,202406,2024-06,20,20,True,0.0


In [ ]:
df_target['has_active_contract'].value_counts()

has_active_contract
True     78452
False     2112
Name: count, dtype: int64

Puede que sea un periodo de prueba gratuito, ya que siempre es en los primeros meses. Lo mas probable es que en las filas que tienen esto nunca haga churn y es lo que aprendera el modelo. Creo que no hay problema con los datos de has_Active_contract = False y lo podemos dejar tal cual esta

### Caso 2: Usuarios que dejan de tener actividad pero no tienen churn confirmada en df_withdrawals entre 2023 y 2024

In [ ]:
#Caso 2: Ejemplo

df_target[df_target['advertiser_zrive_id'] == 17]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
11,17,202301,2023-01,15,35,True,0.0
3563,17,202302,2023-02,14,35,True,0.0
6958,17,202303,2023-03,0,35,True,0.0


In [ ]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 17]

,advertiser_zrive_id,predict_month,churn
11367,17,2022-11,1


En este caso intuyo que el churn es en 2022-11 (fila que tendria churn=1, pero que como es de 2022 no la tenemos) y el resto de filas son la actividad del contrato que ya la  tenia pagado, pero que eliminamos con el enfoque de "eliminar las filas que hay despues de la fila con churn=1"

In [36]:
df_target[df_target['advertiser_zrive_id'] == 46]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
34,46,202301,2023-01,100,100,True,0.0


In [37]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 46]

,advertiser_zrive_id,predict_month,churn
8009,46,2021-04,1
11865,46,2022-12,1


In [40]:
df_target[df_target['advertiser_zrive_id'] == 176]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
136,176,202301,2023-01,11,20,True,0.0
3682,176,202302,2023-02,11,20,True,0.0


In [41]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 176]

,advertiser_zrive_id,predict_month,churn
3360,176,2018-09,1
6448,176,2020-06,1
9489,176,2022-03,1
10387,176,2022-08,1
11048,176,2022-10,1


In [42]:
df_target[df_target['advertiser_zrive_id'] == 18]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
12,18,202301,2023-01,1,50,True,0.0
3564,18,202302,2023-02,1,50,True,0.0
6959,18,202303,2023-03,1,50,False,0.0


In [43]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 18]

,advertiser_zrive_id,predict_month,churn
11471,18,2022-11,1


En estos casos ocurre lo mismo

In [ ]:
df_target[df_target['advertiser_zrive_id'] == 19]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
13827,19,202305,2023-05,10,50,True,0.0
17230,19,202306,2023-06,15,50,True,0.0
20602,19,202307,2023-07,15,50,True,0.0


In [ ]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 19]

,advertiser_zrive_id,predict_month,churn
1354,19,2015-09,1
3068,19,2018-04,1


Pero hay otros casos que no son asi, realmente son churn, pero no queda registrada, entiendo que es a lo que nos referimos en el doc de google de " Deja de haber actividad a partir de cierto momento". Aqui el problema es que sabemos cuando se le acabo el contrato pero no cuando fue el churn, el enfoque que hagamos va a ser una aproximacion.  Ver a cuantos usuarios afecta este problema

In [32]:
user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
# 1. Identificar usuarios sin churn
users_without_churn_ids = user_churn_status[user_churn_status['churn'] == 0]['advertiser_zrive_id'].tolist()

# 2. Analizar el último período registrado para cada usuario sin churn
last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
last_period_no_churn = last_period_by_user[last_period_by_user['advertiser_zrive_id'].isin(users_without_churn_ids)]

# 3. Distribución resumida de los últimos períodos para usuarios sin churn
latest_period = df_target['period_int'].max()
users_active_until_end = last_period_no_churn[last_period_no_churn['period_int'] == latest_period].shape[0]
early_ending_users = last_period_no_churn[last_period_no_churn['period_int'] < latest_period].shape[0]

print(f"Usuarios sin churn activos hasta el último período ({latest_period}): {users_active_until_end}")
print(f"Usuarios sin churn que terminaron antes del último período: {early_ending_users}")

# 4. Verificar el estado del contrato en el último registro para algunos usuarios que terminaron antes
early_ending_sample = last_period_no_churn[last_period_no_churn['period_int'] < latest_period]['advertiser_zrive_id'].head(5).tolist()
early_ending_details = []

for user_id in early_ending_sample:
    last_record = df_target[df_target['advertiser_zrive_id'] == user_id].sort_values('period_int').iloc[-1]
    early_ending_details.append({
        'advertiser_zrive_id': user_id,
        'last_period': last_record['period_int'],
        'has_active_contract': last_record['has_active_contract']
    })

#print("\nEjemplos de usuarios sin churn que terminaron antes:")
#print(pd.DataFrame(early_ending_details))

# 3. Distribución de los últimos períodos para usuarios sin churn
print("\n\n")
last_period_distribution = last_period_no_churn['period_int'].value_counts().sort_index()

print("Distribución de últimos períodos para usuarios sin churn:")
print(last_period_distribution)

Usuarios sin churn activos hasta el último período (202412): 2109
Usuarios sin churn que terminaron antes del último período: 563



Distribución de últimos períodos para usuarios sin churn:
period_int
202301     237
202302      85
202303      61
202304      20
202305      20
202306      11
202307      19
202308      10
202309       5
202310       7
202311      10
202312       7
202401      11
202402       5
202403       3
202404       2
202405       8
202406       8
202407       4
202408       7
202409      11
202410       8
202411       4
202412    2109
Name: count, dtype: int64


Tiene sentido que en los meses que mas ocurra esto sea en los de inicio de 2023, ya que seran casos similares al id 17, 46 o 176 que veiamos arriba. Sin embargo hay casos como el del id 19 que no tienen esa logica

Opciones:
- Eliminar estos casos ya que, la mayoria son realmente churns de finales de 2022 y esta es la actividad posterior que eliminamos con el enfoque propuesto.
En el resto de meses ocurre pero en muy pocos casos, creo que no merece la pena definir un nuevo enfoque para incluir tan pocos casos

### Caso 3: Usuarios con multiples churn

In [ ]:
# Número usuarios únicos
total_users = df_target['advertiser_zrive_id'].nunique()
print(f"Total de usuarios únicos: {total_users}")

# Usuarios sin churn y con churn
user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
users_without_churn = user_churn_status[user_churn_status['churn'] == 0].shape[0]
users_with_churn = user_churn_status[user_churn_status['churn'] == 1].shape[0]
print(f"Usuarios sin churn: {users_without_churn}")
print(f"Usuarios con al menos un churn: {users_with_churn}")

# Usuarios con uno o múltiples churns
churn_counts = df_target[df_target['churn'] == 1].groupby('advertiser_zrive_id').size()
users_with_one_churn = (churn_counts == 1).sum()
users_with_multiple_churn = (churn_counts > 1).sum()
print(f"Usuarios con exactamente un churn: {users_with_one_churn}")
print(f"Usuarios con múltiples churns: {users_with_multiple_churn}")

Total de usuarios únicos: 6534
Usuarios sin churn: 2672
Usuarios con al menos un churn: 3862
Usuarios con exactamente un churn: 2548
Usuarios con múltiples churns: 1314


In [ ]:
df_target[df_target['advertiser_zrive_id'] == 14]

,advertiser_zrive_id,period_int,month_period,monthly_published_ads,monthly_contracted_ads,has_active_contract,churn
36892,14,202312,2023-12,50,50,True,0.0
40124,14,202401,2024-01,50,50,True,1.0
43341,14,202402,2024-02,50,70,True,0.0
46603,14,202403,2024-03,45,70,True,1.0
49929,14,202404,2024-04,0,70,True,0.0


Como adaptar el enfoque de "eliminar las filas que hay despues de la fila con churn=1" en estos casos?. 
Quizas se pueda seguir con ese enfoque, pero  añadiendo una columna que sea "numero de churns historicas" o "churn previo" si o no,  y capturar esa info de esta forma (?), habria que tener cuidado y ver que no hacemos data leakage si entrenamos por execution_month

### Caso 1: Solución

Que haya filas con has_Active_contract en False no es un problema, no hay que hacer nada con los datos

### Caso 2: Solución

Creo que lo que mas sentido tiene es eliminar las filas de los 563 usuarios que en su ultimo mes (sin contar 202412), no tienen la etiqueta de Churn=1. Aunque parezcan muchos usuarios, entre enero,febrero y marzo de 2023, suman 383, los cuales son, siguiendo la logica vista, casos de churn a finales de 2022 y lo que vemos es la actividad posterior (no nos interesa en el modelo)

In [45]:
def remove_inactive_users_without_churn(df_target):
    """
    Elimina las filas de usuarios que terminan su actividad antes del último período
    sin tener un churn registrado.
    
    Args:
        df_target: DataFrame con datos mensuales y etiqueta de churn
    
    Returns:
        DataFrame limpio sin estos usuarios
    """
    # Identificar el último período disponible
    latest_period = df_target['period_int'].max()
    
    # Identificar usuarios sin churn
    user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
    users_without_churn = user_churn_status[user_churn_status['churn'] == 0]['advertiser_zrive_id'].tolist()
    
    # Identificar el último período registrado para cada usuario sin churn
    last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
    
    # Identificar usuarios sin churn que terminan antes del último período
    early_ending_users = last_period_by_user[
        (last_period_by_user['advertiser_zrive_id'].isin(users_without_churn)) & 
        (last_period_by_user['period_int'] < latest_period)
    ]['advertiser_zrive_id'].tolist()
    
    # Eliminar estos usuarios del dataset
    df_clean = df_target[~df_target['advertiser_zrive_id'].isin(early_ending_users)]
    
    return df_clean

In [46]:
df_target = remove_inactive_users_without_churn(df_target)

In [ ]:
user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
# 1. Identificar usuarios sin churn
users_without_churn_ids = user_churn_status[user_churn_status['churn'] == 0]['advertiser_zrive_id'].tolist()

# 2. Analizar el último período registrado para cada usuario sin churn
last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
last_period_no_churn = last_period_by_user[last_period_by_user['advertiser_zrive_id'].isin(users_without_churn_ids)]

# 3. Distribución resumida de los últimos períodos para usuarios sin churn
latest_period = df_target['period_int'].max()
users_active_until_end = last_period_no_churn[last_period_no_churn['period_int'] == latest_period].shape[0]
early_ending_users = last_period_no_churn[last_period_no_churn['period_int'] < latest_period].shape[0]

print(f"Usuarios sin churn activos hasta el último período ({latest_period}): {users_active_until_end}")
print(f"Usuarios sin churn que terminaron antes del último período: {early_ending_users}")

# 4. Verificar el estado del contrato en el último registro para algunos usuarios que terminaron antes
early_ending_sample = last_period_no_churn[last_period_no_churn['period_int'] < latest_period]['advertiser_zrive_id'].head(5).tolist()
early_ending_details = []

for user_id in early_ending_sample:
    last_record = df_target[df_target['advertiser_zrive_id'] == user_id].sort_values('period_int').iloc[-1]
    early_ending_details.append({
        'advertiser_zrive_id': user_id,
        'last_period': last_record['period_int'],
        'has_active_contract': last_record['has_active_contract']
    })

#print("\nEjemplos de usuarios sin churn que terminaron antes:")
#print(pd.DataFrame(early_ending_details))

# 3. Distribución de los últimos períodos para usuarios sin churn
print("\n\n")
last_period_distribution = last_period_no_churn['period_int'].value_counts().sort_index()

print("Distribución de últimos períodos para usuarios sin churn:")
print(last_period_distribution)

Usuarios sin churn activos hasta el último período (202412): 2109
Usuarios sin churn que terminaron antes del último período: 0



Distribución de últimos períodos para usuarios sin churn:
period_int
202412    2109
Name: count, dtype: int64


### Caso 3: Solución